In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from opensoundscape import BoxedAnnotations, CNN
from opensoundscape import SpectrogramPreprocessor
import opensoundscape

In [2]:
#
notebook_name = 'CNN_model_reduce_resampling.ipynb'
#
save_path= '/mnt/class_data/group1_bioacoustics/sheila/model_checkpoints/model_10_reduce_resampling'
#
current_script_path = '/home/Sheila/rewilding_madagascar/CNN_model_reduce_resampling.ipynb'
#
target_path = '/mnt/class_data/group1_bioacoustics/sheila/model_checkpoints/CNN_model_reduce_resampling.ipynb'
#
epochs = 30
#
log_interval=20
#
batch_size=256
#
n_samples_per_class = 1000
#
sample_duration=2
#
window_samples = 2048
#
min_f=0
#
max_f=9000
#
step_size = 10
#
gamma = 0.5
#
update_labels=True






In [3]:
#import labels from csv
train_labels = pd.read_csv('data/train_labels.csv',index_col=['file', 'start_time', 'end_time'])
val_labels = pd.read_csv('data/val_labels.csv',index_col=['file', 'start_time', 'end_time'])
test_labels = pd.read_csv('data/test_labels.csv',index_col=['file', 'start_time', 'end_time'])
train_labels.head()


Hypsipetes_madagascariensis  \
file                                               start_time end_time                                
/mnt/class_data/group1_bioacoustics/sheila/cv4e... 0.0        2.0                             False   
                                                   2.0        4.0                             False   
                                                   4.0        6.0                             False   
                                                   6.0        8.0                             False   
                                                   8.0        10.0                            False   

                                                                        Copsychus_albospecularis  \
file                                               start_time end_time                             
/mnt/class_data/group1_bioacoustics/sheila/cv4e... 0.0        2.0                          False   
                                                   2.0        4.0                          False   
                                                   4.0        6.0                          False   
                                                   6.0        8.0                          False   
                                                   8.0        10.0                         False   

                                                                        Coracopsis_nigra  \
file                                               start_time end_time                     
/mnt/class_data/group1_bioacoustics/sheila/cv4e... 0.0        2.0                  False   
                                                   2.0        4.0                  False   
                                                   4.0        6.0                  False   
                                                   6.0        8.0                  False   
                                                   8.0        10.0                 False   

                                                                        Dicrurus_forficatus  \
file                                               start_time end_time                        
/mnt/class_data/group1_bioacoustics/sheila/cv4e... 0.0        2.0                     False   
                                                   2.0        4.0                     False   
                                                   4.0        6.0                     False   
                                                   6.0        8.0                     False   
                                                   8.0        10.0                    False   

                                                                        Coua_caerulea  \
file                                               start_time end_time                  
/mnt/class_data/group1_bioacoustics/sheila/cv4e... 0.0        2.0               False   
                                                   2.0        4.0               False   
                                                   4.0        6.0               False   
                                                   6.0        8.0               False   
                                                   8.0        10.0              False   

                                                                        Zosterops_maderaspatanus  \
file                                               start_time end_time                             
/mnt/class_data/group1_bioacoustics/sheila/cv4e... 0.0        2.0                          False   
                                                   2.0        4.0                          False   
                                                   4.0        6.0                          False   
                                                   6.0        8.0                          False   
                                                   8.0        10.0                         False   

             

In [4]:
# pick classes to train the model on. These should occur in the annotated data
class_list = ['Hypsipetes_madagascariensis','Copsychus_albospecularis','Coracopsis_nigra','Dicrurus_forficatus','Coua_caerulea','Zosterops_maderaspatanus','Eurystomus_glaucurus','Agapornis_canus','Saxicola_torquatus','Cyanolanius_madagascarinus','Leptopterus_chabert','Nesoenas_picturatus','Coua_reynaudii','Ceblepyris_cinereus','Neodrepanis_coruscans','Philepitta_castanea','Eulemur_sp','Coua_cristata','Treron_australis']

class_list==train_labels.columns.values


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [5]:
train_labels_resampled = opensoundscape.data_selection.resample(train_labels,n_samples_per_class = n_samples_per_class, downsample = False,upsample = True)

In [6]:
import wandb
'''wandb.login()'''#when logging in on a new machine
wandb_session = wandb.init(
    project='Rewilding_Madagascar',
)

wandb: Currently logged in as: sheila-holmes (sheila-holmes-swedish-university-of-agricultural-sciences). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [ ]:
#create new model and change parameters then visualize
# create a CNN and train on the labeled data
model = CNN(architecture='resnet18', sample_duration=sample_duration, classes=class_list)


In [ ]:
'''#Include a step in my preprocessing that does the overlay - it will pick a sample from overlay df and combine with sample currently pre-processing'''
my_preprocessor = SpectrogramPreprocessor(sample_duration = sample_duration, overlay_df=train_labels)
model.preprocessor = my_preprocessor

In [ ]:

#changes learning rate over time
#default is wait a certain number of epochs and then multiply by x (step learning)
model.lr_scheduler_params['kwargs']['step_size'] = step_size
model.lr_scheduler_params['kwargs']['gamma'] = gamma

In [ ]:
model.preprocessor.pipeline.overlay.set(update_labels=update_labels)
model.preprocessor.pipeline.bandpass.set(min_f=min_f,max_f=max_f)
model.preprocessor.pipeline.to_spec.params.window_samples = window_samples
model.train(train_labels_resampled, val_labels, epochs=epochs, log_interval=log_interval, num_workers=8, batch_size=batch_size, wandb_session = wandb_session, save_path= save_path)
wandb_session.finish()

In [ ]:
import shutil
import os

# Use shutil.copyfile to copy the notebook
try:
    shutil.copyfile(current_script_path, target_path)
    print(f"Notebook copied successfully to {target_path}")
except Exception as e:
    print(f"An error occurred while copying the file: {e}")



Notebook copied successfully to /mnt/class_data/group1_bioacoustics/sheila/model_checkpoints/CNN_model_overlay.ipynb
